In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import PIL
import time
from os import listdir
from os.path import isfile, join
import torch.nn.functional as F
from tqdm import tqdm_notebook
import util
filelist = [f for f in listdir('D:/DataSet/AerialImageDataset/train/images') if isfile(join('D:/DataSet/AerialImageDataset/train/images', f))]
labels_path = 'D:/DataSet/AerialImageDataset/train/gt/'
images_path = 'D:/DataSet/AerialImageDataset/train/images/'
plt.rcParams["figure.figsize"] = (12,8)
densenet = models.densenet121(pretrained=True)
densenet
torch.__version__

'0.2.1+a4fc05a'

In [ ]:

class Dense_Net_Feature(nn.Module):
    def __init__(self):
        super(Dense_Net_Feature, self).__init__()
        
        
        self.layer0 = nn.Sequential(densenet.features.conv0, densenet.features.norm0, densenet.features.relu0, densenet.features.pool0)
        self.denseblock1 = densenet.features.denseblock1
        self.transition1 = densenet.features.transition1
        self.denseblock2 = densenet.features.denseblock2
        self.transition2 = densenet.features.transition2
        self.denseblock3 = densenet.features.denseblock3
        self.transition3 = densenet.features.transition3
        self.denseblock4 = densenet.features.denseblock4
        self.final = nn.Sequential(
            nn.Conv2d(43, 20, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
#             nn.Dropout(0.1),
            nn.Conv2d(20, 2, kernel_size=1)
        )
        
        self.reducedim0 = nn.Conv2d(256,10,kernel_size=3,padding=1,bias=False) 
        self.reducedim1 = nn.Conv2d(512,10,kernel_size=3,padding=1,bias=False) 
        self.reducedim2 = nn.Conv2d(1024,10,kernel_size=3,padding=1,bias=False) 
        self.reducedim3 = nn.Conv2d(1024,10,kernel_size=3,padding=1,bias=False) 
#         self.ppm = _PyramidPoolingModule(512, 512, (1, 2, 3, 6))
#         self.final = nn.Sequential(
#             nn.Conv2d(2560, 512, kernel_size=3, padding=1, bias=False),
#             nn.BatchNorm2d(512, momentum=.95),
#             nn.ReLU(inplace=True),
#             nn.Dropout(0.1),
#             nn.Conv2d(512, num_classes, kernel_size=1)
#         )

#         if use_aux:
#             self.aux_logits = nn.Conv2d(1024, num_classes, kernel_size=1)
#             initialize_weights(self.aux_logits)

#         initialize_weights(self.ppm, self.final)

    def forward(self, x):
        x_size = x.size()
        
        out = self.layer0(x)
        
        
        out = self.denseblock1(out)
        upsample0 = F.upsample(self.reducedim0(out),x_size[2:],mode='bilinear')
        out = self.transition1(out)
        

        out = self.denseblock2(out)
        upsample1 = F.upsample(self.reducedim1(out),x_size[2:],mode='bilinear')
        out = self.transition2(out)
        
        
        out = self.denseblock3(out)
        upsample2 = F.upsample(self.reducedim2(out),x_size[2:],mode='bilinear')
        out = self.transition3(out)
        out = self.denseblock4(out)
        upsample3 = F.upsample(self.reducedim3(out),x_size[2:],mode='bilinear')
        
        out = self.final(torch.cat((x,upsample0,upsample1,upsample2,upsample3),1))
        
        return out
#         x = self.layer0(x)
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         if self.training and self.use_aux:
#             aux = self.aux_logits(x)
#         x = self.layer4(x)
#         print(x.size())
#         x = self.ppm(x)
#         x = self.final(x)
#         if self.training and self.use_aux:
#             return F.upsample(x, x_size[2:], mode='bilinear'), F.upsample(aux, x_size[2:], mode='bilinear')
#         return F.upsample(x, x_size[2:], mode='bilinear')

In [ ]:
densenet_f = Dense_Net_Feature()
CROP_SIZE =500
# densenet_f = densenet_f.cuda()
# densenet_f

### if image contain too little one label,250000*5% just skip

In [ ]:
def train_model(model ,criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0
    best_loss = np.inf
    for epoch in range(num_epochs):
        rand_sample = np.random.randint(len(filelist), size=20)
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        
            
        #scheduler.step()
        model.train(True)  # Set model to training mode
        running_loss = 0.0
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        out_transform = transforms.Compose([
                transforms.ToTensor(),
#                 normalize
        ])
        out_transform_lebel = transforms.Compose([
                transforms.ToTensor()
#                 normalize
        ])
        # Iterate over data.
        count = 0
        cum = 0
        for index in tqdm_notebook(rand_sample,total=20):
            im = PIL.Image.open(join(images_path,filelist[index]))
            labels_data = PIL.Image.open(join(labels_path,filelist[index]))
            
            train_data = out_transform(im)
            labels_data = out_transform_lebel(labels_data)
            im.close()
            for i in range(5000//CROP_SIZE):
                for j in range(5000//CROP_SIZE):
                    
                    labels_data_tem = Variable(labels_data[:,i*CROP_SIZE:i*CROP_SIZE+CROP_SIZE,j*CROP_SIZE:j*CROP_SIZE+CROP_SIZE].type(torch.LongTensor).cuda())
                    cum = torch.sum(labels_data_tem).cpu().data.numpy()[0]
                    if cum < CROP_SIZE*CROP_SIZE*0.04:
                        continue
                    count+=1
                    train_data_tem = train_data[:,i*CROP_SIZE:i*CROP_SIZE+CROP_SIZE,j*CROP_SIZE:j*CROP_SIZE+CROP_SIZE].unsqueeze(0)
                    train_data_tem = Variable(train_data_tem.cuda())
                    
# #                     print(train_data_tem.size())
                    
                    # zero the parameter gradients
        
                    # forward
                    outputs= model(train_data_tem)
#                     print(labels_data.size())
                    loss = criterion(outputs,labels_data_tem) 
                    
                    
        
                    #loss = MSE(outputs, inputs_eva)
                    # backward + optimize only if in training phase
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    # statistics
                    running_loss += loss.data[0]
        epoch_loss = running_loss / count
        if(epoch_loss<best_loss):
            best_model_wts = model.state_dict()
            best_loss = epoch_loss
        print('Loss: {:.4f} Iteration:{}'.format(epoch_loss,count))
    # deep copy the model
    

    print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    # load best model weights
#     model.load_state_dict(best_model_wts)
    return model

In [ ]:
# labels_data = PIL.Image.open(join(labels_path,filelist[0]))
# out_transform = transforms.Compose([
#     transforms.ToTensor()
# ])
# # train_data = out_transform(im)
# labels_data = out_transform(labels_data)
# labels_data.size()

In [ ]:
# densenet_f.load_state_dict(torch.load('vnew0.pt'))
# use_gpu = torch.cuda.is_available()
# if use_gpu:
#     densenet_f = densenet_f.cuda()
criterion = util.CrossEntropyLoss2d(size_average=True).cuda()
optimizer_ft = optim.Adam(densenet_f.parameters(), lr=0.000001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
densenet_f = train_model(densenet_f,criterion,optimizer_ft,exp_lr_scheduler,10)

In [ ]:
# torch.save(densenet_f.state_dict(),'vnew5.pt')
densenet_f.load_state_dict(torch.load('vnew5.pt'))
use_gpu = torch.cuda.is_available()
if use_gpu:
    densenet_f = densenet_f.cuda()

In [ ]:
-

In [ ]:
total_acc/(5*5*len(test_list))

In [ ]:
label_im = PIL.Image.open(labels_path+filelist[0])
label_im = np.asarray(label_im)
label_im = label_im==255

label_im2 = PIL.Image.open(labels_path+filelist[2])
label_im2 = np.asarray(label_im2)
label_im2 = label_im2==255

plt.imshow(label_im)
plt.show()

plt.imshow(label_im2)
plt.show()
(np.logical_and(label_im,label_im2)*1).sum()

In [ ]:
test_path='fuckcity.tif'
test_transform = transforms.Compose([
    transforms.ToTensor()
])
test = test_transform(PIL.Image.open(test_path)).unsqueeze(0)
test = Variable(test.cuda(),volatile=True)
densenet_f.eval()
out = densenet_f(test)
m = nn.Softmax2d()
out = m(out)
out = (out.cpu()[0].data.numpy()).transpose((1, 2, 0))



f,ax = plt.subplots(1,2)

ax[0].imshow(out[:,:,1])
ax[1].imshow(np.array(PIL.Image.open(test_path)))
plt.show()

